In [1]:
#imports
import pandas as pd
import numpy as np

Get Data from external sources

In [2]:
from sklearn import datasets
iris = datasets.load_iris()
print(type(iris))
X, y = iris.data, iris.target

<class 'sklearn.datasets.base.Bunch'>


In [43]:
def fnHandleMsg(msg,msgType=""):
    print(msg)


def fnAssignColnames(df,colNames=None):
    if colNames is not None and len(colNames)==len(df.columns):
        df.columns=colNames

    return df

    
    
def fnCreatePdFrameFromArray(arrayTuple,colNames=None):
    #return pd.DataFrame(np.column_stack(arrayTuple))
    df=None
    for a in arrayTuple:
        if len(a.shape)==1:
            n=pd.Series(a)
        else:
            n=pd.DataFrame(a)
        if df is None:
            df=pd.DataFrame(n)
        else:
            df=pd.concat([df,n],axis=1)

    return df

def fnGetDf(data,colNames=None):
    df=fnCreatePdFrameFromArray(data,colNames)
    df=fnAssignColnames(df,colNames)
    return df
                        
def fnFilterMissingValues(df):
    missingValues=df.isnull().sum(axis=1)
    missingValuesCount=sum(missingValues>0)
    fnHandleMsg("No. of Missing Values in DF : "+str(missingValuesCount))
    df=df.dropna()
    return df,missingValuesCount

def fnGetUniqueValLenColumns(df):
    iDfLen=len(df)
    lUniqueValCount=[]
    for col in df.columns:
        iUniqueValLen=len(np.unique(df[col]))
        lUniqueValCount.append(iUniqueValLen)
        
    return lUniqueValCount


def fnNormalizeCols(df,colNames):
    fnHandleMsg("Normalizing Values in DF for columns: "+str(colNames))
    from sklearn import preprocessing
    dfNew = pd.DataFrame(df[colNames])
    min_max_scaler = preprocessing.MinMaxScaler()
    np_scaled = min_max_scaler.fit_transform(dfNew)
    df_normalized = pd.DataFrame(np_scaled)
    df_normalized.columns=colNames
    df[colNames]=df_normalized
    #for col in colNames:
        #df[col]=df_normalized[col]
        
    return df

def fnPlotCols(df,colNames=None,num_bins=50):
    import matplotlib.pyplot as plt

    if colNames is None:
        colNames=df.columns
        
    for colName in colNames:
        colData=df[colName]
        fnPlotHistogram(colData,colName)
        fnPlotBoxplot(colData,colName)
        

def fnPlotHistogram(colData,colName,num_bins=50):
    mu=np.mean(colData)
    sigma=np.std(colData)


    fig, ax = plt.subplots()

    # the histogram of the data
    n, bins, patches = ax.hist(colData, num_bins, normed=True)

    # add a 'best fit' line
    y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
    ax.plot(bins, y, '--')
    ax.set_xlabel('Smarts')
    ax.set_ylabel('Probability density')
    ax.set_title(r'Histogram of {0}: mu={1}, sigma={2}'.format(colName,mu,sigma))

    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()
    plt.show()

def fnPlotBoxplot(colData,colName):
    colData=df[colName]
    fig1, ax1 = plt.subplots()
    ax1.set_title(r'Box Plot of {0}'.format(colName))
    ax1.boxplot(colData)

    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()
    plt.show()

    
def RandomizeDF(df,seed=300):    
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    df=df.loc[perm]
    df.reset_index(inplace=True)
    return df

def GetXyDf(df,outputVarName):
    y=df[outputVarName]
    X=df.loc[:, df.columns != outputVarName]
    return X,y
    
    
    
def GetTrainTest(df,outputVarName,testPer=0.2,seed=200):
    from sklearn.model_selection import train_test_split
    X,y=GetXyDf(df,outputVarName)
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=testPer)
    
    X_train.reset_index(inplace=True,drop=True)
    X_test.reset_index(inplace=True,drop=True)
    y_train.reset_index(inplace=True,drop=True)
    y_test.reset_index(inplace=True,drop=True)
    
    return X_train, X_test, y_train, y_test
    
    
    
def GetkFoldedTrainVal(X,y,nFolds=5,randomstate=300):    
    from sklearn.model_selection import StratifiedKFold
    skf = StratifiedKFold(n_splits=nFolds)
    
    for train_index, test_index in skf.split(X, y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        yield X_train, X_test ,y_train, y_test
            
            
def GetScores(actualY,Pred,categoriesNames):
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import f1_score
    from sklearn.metrics import precision_recall_fscore_support
    from sklearn.metrics import classification_report
    acc_score=accuracy_score(actualY, Pred,normalize=True)
    target_names = categoriesNames
    f1score=classification_report(actualY, Pred, target_names=target_names)
    cnf_matrix = confusion_matrix(actualY, Pred)
    return acc_score,f1score,cnf_matrix



    
            
    


Convert Data into Pandas Dataframe

In [4]:
outputVarName='fType'
df=fnGetDf((X,y),("SL","SH","PL","PH",outputVarName))

print(df.head())

    SL   SH   PL   PH  fType
0  5.1  3.5  1.4  0.2      0
1  4.9  3.0  1.4  0.2      0
2  4.7  3.2  1.3  0.2      0
3  4.6  3.1  1.5  0.2      0
4  5.0  3.6  1.4  0.2      0


DataSet Exploration

In [5]:
print(df.describe())

               SL          SH          PL          PH       fType
count  150.000000  150.000000  150.000000  150.000000  150.000000
mean     5.843333    3.054000    3.758667    1.198667    1.000000
std      0.828066    0.433594    1.764420    0.763161    0.819232
min      4.300000    2.000000    1.000000    0.100000    0.000000
25%      5.100000    2.800000    1.600000    0.300000    0.000000
50%      5.800000    3.000000    4.350000    1.300000    1.000000
75%      6.400000    3.300000    5.100000    1.800000    2.000000
max      7.900000    4.400000    6.900000    2.500000    2.000000


In [6]:
df,missingValuesCount=fnFilterMissingValues(df)

No. of Missing Values in DF : 0


In [7]:
lUniqueValCount=fnGetUniqueValLenColumns(df)
print(list(zip(lUniqueValCount,df.dtypes)))

[(35, dtype('float64')), (23, dtype('float64')), (43, dtype('float64')), (22, dtype('float64')), (3, dtype('int32'))]


In [8]:
print(df.describe())
colNames=["SH","SL","PL","PH"]
df=fnNormalizeCols(df,colNames)
print(df.describe())

               SL          SH          PL          PH       fType
count  150.000000  150.000000  150.000000  150.000000  150.000000
mean     5.843333    3.054000    3.758667    1.198667    1.000000
std      0.828066    0.433594    1.764420    0.763161    0.819232
min      4.300000    2.000000    1.000000    0.100000    0.000000
25%      5.100000    2.800000    1.600000    0.300000    0.000000
50%      5.800000    3.000000    4.350000    1.300000    1.000000
75%      6.400000    3.300000    5.100000    1.800000    2.000000
max      7.900000    4.400000    6.900000    2.500000    2.000000
Normalizing Values in DF for columns: ['SH', 'SL', 'PL', 'PH']
               SL          SH          PL          PH       fType
count  150.000000  150.000000  150.000000  150.000000  150.000000
mean     0.428704    0.439167    0.467571    0.457778    1.000000
std      0.230018    0.180664    0.299054    0.317984    0.819232
min      0.000000    0.000000    0.000000    0.000000    0.000000
25%      0.22

In [9]:
#fnPlotCols(df,colNames)

In [35]:
print(df[outputVarName].value_counts())
X_trainVal, X_test, y_trainVal, y_test=GetTrainTest(df,outputVarName)
print(y_trainVal.value_counts())
print(y_test.value_counts())

2    50
1    50
0    50
Name: fType, dtype: int64
2    40
1    40
0    40
Name: fType, dtype: int64
2    10
1    10
0    10
Name: fType, dtype: int64


In [46]:
results=[]
for X_train, X_val ,y_train, y_val in GetkFoldedTrainVal(X_trainVal,y_trainVal):
    """
    print(len(X_train),len(X_val))
    print(y_train.value_counts())
    print(y_val.value_counts())\
    """
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
    clf.fit(X_train, y_train)
    predClass=clf.predict(X_val)
    
    categoriesNames=[str(x) for x in np.unique(y_val)]
    acc_score,f1score,cnf_matrix=GetScores(y_val,predClass,categoriesNames)
    results.append((acc_score,f1score,cnf_matrix))
    print(acc_score,f1score,cnf_matrix)
    


0.875              precision    recall  f1-score   support

          0       1.00      1.00      1.00         8
          1       0.78      0.88      0.82         8
          2       0.86      0.75      0.80         8

avg / total       0.88      0.88      0.87        24
 [[8 0 0]
 [0 7 1]
 [0 2 6]]
0.9583333333333334              precision    recall  f1-score   support

          0       1.00      1.00      1.00         8
          1       1.00      0.88      0.93         8
          2       0.89      1.00      0.94         8

avg / total       0.96      0.96      0.96        24
 [[8 0 0]
 [0 7 1]
 [0 0 8]]
0.9166666666666666              precision    recall  f1-score   support

          0       1.00      1.00      1.00         8
          1       1.00      0.75      0.86         8
          2       0.80      1.00      0.89         8

avg / total       0.93      0.92      0.92        24
 [[8 0 0]
 [0 6 2]
 [0 0 8]]
0.8333333333333334              precision    recall  f1-score   supp

In [48]:
print("Mean Accuracy : ",np.mean([x[0] for x in results]))
print("Accuracy Variance: ",np.std([x[0] for x in results]))

Mean Accuracy :  0.9166666666666667
Accuracy Variance:  0.05892556509887895


Thus the baseline accuacy for this Dataset is 91% with very slight variation.